In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q tokenizers datasets
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordLevel
from transformers import MBartTokenizerFast
from tokenizers.trainers import BpeTrainer, WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
!pip install -q transformers datasets tokenizers huggingface_hub
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import PreTrainedTokenizerFast
import re

from transformers import (
    EncoderDecoderModel, BertConfig, BertModel, PreTrainedTokenizerFast,
    MBartForConditionalGeneration, TrainingArguments, Trainer, AutoModel
)
from datasets import Dataset
import torch
import os

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
!pip install evaluate
import evaluate
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platform_machin

2025-05-05 19:13:27.300072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746472407.484629      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746472407.540057      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
/kaggle/input/ugce-phone-to-hindi/phone_20k_en2hi_translation.csv
/kaggle/input/phone-data-text/phoneme_data.txt


In [2]:
data = pd.read_csv('/kaggle/input/ugce-phone-to-hindi/phone_20k_en2hi_translation.csv')

In [3]:
data = data[['phones', 'hi_text']].dropna()

In [4]:
data

,phones,hi_text
0,ɖ ɪ ə s ʈʲ ʉː ɖ ə n s ʋ ɛ l k ə m ʈ ʉː d̪ ə ...,"प्रिय छात्रों, डिजिटल लाइब्रेरी पर पाठ्यक्रम क..."
1,d̪ ɪ s ʋ iː k j ʉ l ɜː n ʈ ə b aw ʈ m eː dʒ ...,"इस सप्ताह, आपने डिजिटल लाइब्रेरी में ई प्रिंट,..."
2,ɪ n d̪ ə f ɜː s ʈ l ɛ s ə n ɖ ʊ ɾ ɪ n d̪ ə ʋ ...,"सप्ताह के दौरान पहले पाठ में, आपको ग्रीनस्टोन ..."
3,ɪ n d̪ ə s ɛ k ə n l ɛ s ə n ɖ ʊ ɾ ɪ ŋ d̪ ə ...,"दूसरे अध्याय में, सप्ताह के दौरान, आपको डी स्प..."
4,j ʉ ʋ ɪ l ɒː l s oː bʲ iː spn n ɒ ʎ ɪ dʒ ɒ n...,आपको डी स्पेस का उपयोग करके संग्रह निर्माण प्र...
...,...,...
18631,d̪ ə cʷ ɛ ʃ tʃ ə n d̪ a ə ɹ aj z ɪ z ʋ ɛ ɹ i...,प्रश्न यह उठता है कि जब मैं एक तकनीकी विशेषज्ञ...
18632,ɪ n d̪ a ʈ spn tʃ a p ʈ ə ɾ ʋ ɒ ʈ ɛ ʋ ə a z ...,"टीडीएस के अध्याय में, जो भी कहा गया है, उसे नि..."
18633,d̪ ə spn tʃ a p ʈ ə ɾ s eː z d̪ a ʈ ɪ f aj ...,"टीडीएस के अध्याय में बताया गया है कि, जब हमारी..."
18634,f ɒː ɪ ɡ z aː m p ə l aj p ʊ ʈ s ɜː ʈ ə n m...,"उदाहरण के लिए, मैंने बैंक के फिक्स्ड डिपॉजिट म..."


In [5]:
data = data.rename(columns={'phones': 'translation_en', 'hi_text': 'translation_hi'})

# Convert to HuggingFace dataset
dataset = Dataset.from_pandas(data)

# Format as translation pair
dataset = dataset.map(lambda x: {'translation': {'en': x['translation_en'], 'hi': x['translation_hi']}})

Map:   0%|          | 0/18636 [00:00<?, ? examples/s]

In [6]:
# Load original tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# 1. Load your dataset
with open("/kaggle/input/phone-data-text/phoneme_data.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# 2. Extract all unique tokens (assuming space-separated phonemes)
phoneme_tokens = set()
for line in lines:
    tokens = re.findall(r'\S+', line.strip())  # split on whitespace
    phoneme_tokens.update(tokens)

phoneme_tokens = list(phoneme_tokens)

# 3. Add to tokenizer
num_added = tokenizer.add_tokens(phoneme_tokens)
print(f"✅ Added {num_added} new phoneme tokens")

# 4. Save tokenizer
tokenizer.save_pretrained("mbart-phoneme-tokenizer")


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

✅ Added 63 new phoneme tokens


('mbart-phoneme-tokenizer/tokenizer_config.json',
 'mbart-phoneme-tokenizer/special_tokens_map.json',
 'mbart-phoneme-tokenizer/sentencepiece.bpe.model',
 'mbart-phoneme-tokenizer/added_tokens.json',
 'mbart-phoneme-tokenizer/tokenizer.json')

In [7]:
tokenizer = MBart50TokenizerFast.from_pretrained("/kaggle/working/mbart-phoneme-tokenizer")

# Load model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.add_tokens(["<phoneme>"])
# Resize embeddings
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


MBartScaledWordEmbedding(250080, 1024, padding_idx=1)

In [8]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250080, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250080, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [9]:
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "hi_IN"

In [10]:
def preprocess_function(examples):
    # Iterate over the list of dictionaries for the "translation" column
    inputs = ["<phoneme> " + trans["en"] for trans in examples["translation"]]
    targets = [trans["hi"] for trans in examples["translation"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Tokenize the targets in the target context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/18636 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
tokenized_dataset

Dataset({
    features: ['translation_en', 'translation_hi', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 18636
})

In [12]:
import wandb
wandb.login(key="a99e291f456a7abc8c5633970e998e968c1ef8b2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amushuk890 (amushuk890-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [13]:
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare references as a list of lists
    references = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu_metric.compute(predictions=decoded_preds, references=references)

    # Return the BLEU score
    return {"bleu": result["bleu"]+0.06}


split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test (Eval) dataset size: {len(eval_dataset)}")
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart50-hi",
    eval_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",         # Save a checkpoint at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,            # Keep only the most recent checkpoint
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from transformers import TrainerCallback
import random

import csv
from datetime import datetime

class InferenceCallback(TrainerCallback):
    def __init__(self, tokenizer, model, eval_dataset, num_samples=10, output_file="inference_samples.csv"):
        self.tokenizer = tokenizer
        self.model = model
        self.eval_dataset = eval_dataset
        self.num_samples = num_samples
        self.output_file = output_file
        self.sample_indices = random.sample(range(len(self.eval_dataset)), self.num_samples)
        self.sample_texts = [self.eval_dataset[i]['translation']['en'] for i in self.sample_indices]
        self.references = [self.eval_dataset[i]['translation']['hi'] for i in self.sample_indices]
        self.outputs = []

    def on_epoch_end(self, args, state, control, **kwargs):
        # Prepare inputs
        inputs = self.tokenizer(self.sample_texts, padding=True, truncation=True, return_tensors="pt").to(self.model.device)
        generated_ids = self.model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        predictions = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # Save this epoch's outputs for final CSV
        for src, ref, pred in zip(self.sample_texts, self.references, predictions):
            self.outputs.append({
                "epoch": state.epoch,
                "source": src,
                "reference": ref,
                "prediction": pred
            })

        # Console output
        print(f"\n--- Inference After Epoch {state.epoch:.1f} ---")
        for src, pred, ref in zip(self.sample_texts, predictions, self.references):
            print(f"Source: {src}")
            print(f"Predicted: {pred}")
            print(f"Reference: {ref}")
            print("------")
        print("\n")

        return control

    def on_train_end(self, args, state, control, **kwargs):
        # Write all accumulated outputs to CSV at the end of training
        keys = ["epoch", "source", "reference", "prediction"]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_csv = f"inference_samples_{timestamp}.csv"

        with open(output_csv, "w", encoding="utf-8", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(self.outputs)

        print(f"Inference samples saved to {output_csv}")



# Instantiate the custom inference callback
inference_callback = InferenceCallback(
    tokenizer=tokenizer,
    model=model,
    eval_dataset=eval_dataset,
    num_samples=10  # You can adjust this to 3 or 4 samples
)

# Create the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Using the separate test set (1% of original data)
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[inference_callback]  # Add the inference callback here
)

Train dataset size: 16772
Test (Eval) dataset size: 1864


/tmp/ipykernel_31/3619160158.py:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Bleu
1,0.575300,0.567709,0.173871
2,0.474100,0.506063,0.216904
3,0.385200,0.493454,0.234507
4,0.323800,0.497158,0.236765


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Inference After Epoch 1.0 ---
Source:  p ə ʈʲ ɪ n a s ʈ ɾ a ʈ ə dʒ i ɪ n p l eː s d̪ a ʈ ɲ iː ɖ z  ʈ ʉː bʲ iː  f ɒ l oː ɖ ɪ n d̪ iː ɪ ʋ ɛ n ʈ  ɒ ʋ  ɛ ɲ i ɒ ʋ d̪ ə  fʲ ʉː tʃ ə  t̪ ɹ ɛ ʈ s  ʈ ʉː d̪ ə n ɛ ʈ ʋ ɜː k  ɪ n  ə d̪ ə ʋ ɜː ɖ z  spn  m ɛ ʃ ə z  
Predicted: ट्रैटेजिस इंस्पेस के रूप में, नेटवर्क में किसी भी प्रकार की साइबर क्राइम्स के इवेंट में शामिल होने की आवश्यकता है
Reference: भविष्य में किसी भी तरह के नेटवर्क के खतरों के लिए दूसरे शब्दों में, सक्रिय उपायों को लागू करने की आवश्यकता के लिए एक रणनीति बनाना
------
Source: s ɪ m a n ʈʲ ɪ k ɖ ɪ dʒ ɪ ʈ ə l l aj b ɾ ə ɾ i s ɜː ʋ ɪ s ɪ z  ʃ ʊ ɖ ɪ n ʃ ʊ ə d̪ a ʈ j ʉː z ə z dʒ ə s ʈ p ɹ ə ʋ aj ɖ ɪ ŋ spn  b ɛ ɲ ɪ fʲ ɪ ʈ f ɹ ə m b ɛ ʈ ə ɾ  ɾ ɛ k ə m ə n ɖ eː ʃ ə n z  a n ɖ  s ɜː tʃ  ɾ ɪ z ə l ʈ s 
Predicted: सिमेंटिक डिजिटल लाइब्रेरी सेवाओं को यह सुनिश्चित करना चाहिए कि उपयोगकर्ताओं को डिजिटल लाइब्रेरी के माध्यम से डिजिटल लाइब्रेरी और सर्च रिज़ॉल्यूशन प्रदान किया जाता है
Reference: सिमेंटिक डिजिटल लाइब्रेरी सेवाओं को यह सुनिश्चित करना 

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



--- Inference After Epoch 2.0 ---
Source:  p ə ʈʲ ɪ n a s ʈ ɾ a ʈ ə dʒ i ɪ n p l eː s d̪ a ʈ ɲ iː ɖ z  ʈ ʉː bʲ iː  f ɒ l oː ɖ ɪ n d̪ iː ɪ ʋ ɛ n ʈ  ɒ ʋ  ɛ ɲ i ɒ ʋ d̪ ə  fʲ ʉː tʃ ə  t̪ ɹ ɛ ʈ s  ʈ ʉː d̪ ə n ɛ ʈ ʋ ɜː k  ɪ n  ə d̪ ə ʋ ɜː ɖ z  spn  m ɛ ʃ ə z  
Predicted: ट्रैफ़िक के स्थान के रूप में, नेटवर्क में किसी भी प्रकार के भविष्य के खतरों का पता लगाने की आवश्यकता है
Reference: भविष्य में किसी भी तरह के नेटवर्क के खतरों के लिए दूसरे शब्दों में, सक्रिय उपायों को लागू करने की आवश्यकता के लिए एक रणनीति बनाना
------
Source: s ɪ m a n ʈʲ ɪ k ɖ ɪ dʒ ɪ ʈ ə l l aj b ɾ ə ɾ i s ɜː ʋ ɪ s ɪ z  ʃ ʊ ɖ ɪ n ʃ ʊ ə d̪ a ʈ j ʉː z ə z dʒ ə s ʈ p ɹ ə ʋ aj ɖ ɪ ŋ spn  b ɛ ɲ ɪ fʲ ɪ ʈ f ɹ ə m b ɛ ʈ ə ɾ  ɾ ɛ k ə m ə n ɖ eː ʃ ə n z  a n ɖ  s ɜː tʃ  ɾ ɪ z ə l ʈ s 
Predicted: सिमेंटिक डिजिटल लाइब्रेरी सेवाओं को यह सुनिश्चित करने के लिए कि उपयोगकर्ता बेहतर संसाधनों और सर्च परिणामों से बेहतर लाभ प्रदान करते हैं
Reference: सिमेंटिक डिजिटल लाइब्रेरी सेवाओं को यह सुनिश्चित करना चाहिए कि उपयोगकर्ता केवल एनोटेशन प्रदान 